forked Iowa_contiguousdistricting notebook on December 6

Manually implementing algorithm described in Chen and Rodden (2013) in Steps 1-3 in Helper Functions subsection

Shapefile dataframes used in this notebook:
- shapefile_iowa: MGGG stuff merged with census stuff
- shapef_ia_proj: projected to UTM so we can run distance calculations on it
- shapef_ia_fordistricting: 99 rows for each county, ready for initial allocation; made from deep copy of shapef_ia_proj
- shapef_counties_for_realloc: made from a deep copy from shapef_ia_fordistricting; this has a num_switches column and a district column populated with the district assignment from the initial allocation
- shapef_ia_initialdistricting: came from _fordistricting, went through the districting (step 1/2) process, and is now 4 rows
	as of Sept 18, not modified further for step 3/district balancing
- shapef_ia_redist: this is a deep copy of shapef_ia_initialdistricting with a few dropped columns. This df is used for making new districts

In [1]:
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from PIL import Image, ImageOps
import glob
import os       #mkdir
from scipy.sparse import csgraph #for laplacian
from scipy.linalg import null_space
from plotnine import (ggplot, aes, geom_map, geom_text, geom_label,
                     ggtitle, element_blank, element_rect,
                     scale_fill_manual, theme_minimal, theme, scale_fill_cmap)
import math         ##for math.sqrt
import random       #for random selection of district to start with

# Helper Functions

## Importing and cleaning data

In [2]:
#census.csv is data from Secretary of State's office.
census_df=pd.read_csv('census.csv')
census_df['COUNTYFP10']=census_df['COUNTYFP10'].astype(str).str.pad(3,fillchar='0')

#imports county shapefiles from MGGG
shapefile_iowa = gpd.read_file('IA_counties/IA_counties.shp').sort_values('NAME10',ignore_index=True)

## Merging ONLY 2020 population numbers and county_id from census df into shapefile_iowa
shapefile_iowa = shapefile_iowa.merge(census_df[['COUNTYFP10','population','county_id']], on='COUNTYFP10').copy()

county_populations = np.array(census_df['population'])
state_population = sum(county_populations)

#Then project the shapefiles to UTM 15N
shapef_ia_proj = shapefile_iowa.to_crs(epsg=26915)

## Merging ONLY lat/long + county id from census df into shapefile_iowa (since population and county_id are already there)
map_population_by_county_data = shapefile_iowa.merge(census_df[['COUNTYFP10','latitude','longitude']], on='COUNTYFP10').copy()

# adding/fixing columns with (projected) centroid locations 
shapef_ia_proj['xcentr_lon'] = shapef_ia_proj.centroid.x
shapef_ia_proj['ycentr_lat'] = shapef_ia_proj.centroid.y

## cutting out other columns from the shapefile to be dissolved on

districting_columns = ['COUNTYFP10', 'NAME10', 'geometry', 
       'population', 'county_id', 'xcentr_lon', 'ycentr_lat']

#make a new shapefile, which will be merged/dissolved on in the process of making districts
shapef_ia_fordistricting = shapef_ia_proj[districting_columns].copy()

#add column of county indices (which will get concatenated, as county_id_string)
shapef_ia_fordistricting['county_id_string'] = shapef_ia_fordistricting['county_id']
shapef_ia_fordistricting['county_id_string'] = shapef_ia_fordistricting['county_id_string'].astype(str).str.pad(2,fillchar='0')

#add column of county indices (which will become district indices)
shapef_ia_fordistricting['temp_district'] = shapef_ia_fordistricting.index

## Nearest-Neighbor district-building model (adjacency matrix + distance matrix)

In [3]:
# function for arbitrary adjacency matrix
# updated to no longer classify kitty-corner as "adjacent", Oct 30

def adj_mat_calc(temp_shapefile):
    curr_n_districts = len(temp_shapefile)
    adjac_mat = pd.DataFrame()

    for i in range(curr_n_districts):
        adjac_mat[i] = temp_shapefile.intersection(temp_shapefile.iloc[[i]].unary_union).length
    
    #sign function turns all nonzero entries to 1
    adjac_mat = np.sign(adjac_mat)

    #eliminate diagonals, so county i is not adjacent to itself
    adjac_mat = adjac_mat - np.identity(curr_n_districts)

    # make everything an integer
    adjac_mat = adjac_mat.astype(int)

    #and/or boolean?
    # ia_adjac_matrix_bool = ia_adjac_matrix.astype(bool)

    return adjac_mat


#function: input is a shapefile with centroid columns, output is a distance matrix
def temp_distance_matrix(temp_shapefile):
    curr_n_districts = len(temp_shapefile)
    distance_mat = np.zeros((curr_n_districts,curr_n_districts))

    for i in range(curr_n_districts):
        for j in range(i):      #just do half the triangle, so indices from 0 to i-1
            x_dist = (temp_shapefile['xcentr_lon'].iloc[i] - temp_shapefile['xcentr_lon'].iloc[j])
            y_dist = (temp_shapefile['ycentr_lat'].iloc[i] - temp_shapefile['ycentr_lat'].iloc[j])
            distance_mat[i,j] = math.sqrt(x_dist**2 + y_dist**2)
            distance_mat[j,i] = math.sqrt(x_dist**2 + y_dist**2)

    return distance_mat

## Steps 1 and 2 initial allocation

In [4]:
#make sure random seed works okay inside function
def initial_alloc_func(shapef_ia_fordistricting):
    shapef_ia_initialdistricting = shapef_ia_fordistricting.copy()

    for i in range(n_counties - n_districts): # 95 iterations brings us from 99 districts to 4
        #how many districts are we working with this time?
        running_ndistricts = len(shapef_ia_initialdistricting)

        # pick out a district to work on on this iteration of the loop
        running_index = random.randint(0,running_ndistricts-1)        

        #find the temp_district associated with the running index
        #     the below was returning a slice of a dataframe, and not just the entry
        # running_temp_dist = shapef_ia_initialdistricting.loc[shapef_ia_initialdistricting.index == running_index, 'temp_district']
        running_temp_district = shapef_ia_initialdistricting['temp_district'].iloc[running_index]
        #     originally just called this for the print statement
            
        #set up adjacency and distance matrices
        running_adjmat = adj_mat_calc(shapef_ia_initialdistricting)
        running_distmat = temp_distance_matrix(shapef_ia_initialdistricting)

        # print("On loop # %d (with %d districts remaining), we have selected index %d. \
        # \n This corresponds to county id %s and temporary district # %d."   \
        #     % (i+1, running_ndistricts,running_index,   \
        #        shapef_ia_initialdistricting['county_id_string'].iloc[running_index], \
        #        running_temp_district) )

        distance_list = list(running_distmat[running_index])

        neighbor_dist = sorted(distance_list)[1]        #second smallest distance is nearest neighbor (since distance to self is zero)
        neighbor_index = distance_list.index(neighbor_dist)   
        
        # the temp_district number associated with the neighbor_index
        neighbor_temp_district= shapef_ia_initialdistricting['temp_district'].iloc[neighbor_index]
        #   this doesn't really get used except to print? but running_temp_district is super important for re-indexing

        # print("The nearest neighbor index is %d, representing county id %s and temporary district # %d." \
        #     % (neighbor_index,   \
        #        shapef_ia_initialdistricting['county_id_string'].iloc[neighbor_index],\
        #        neighbor_temp_district) )

        # re-index the neighbor county to be in the first county's district
        shapef_ia_initialdistricting.loc[shapef_ia_initialdistricting.index == neighbor_index, 'temp_district'] = running_temp_district      


        # dissolve shapefile based on temp_district to combine the two counties
        #aggregate remaining columns by summing them

        # arguments for aggfunc: https://geopandas.org/en/stable/docs/user_guide/aggregation_with_dissolve.html
        shapef_ia_initialdistricting = shapef_ia_initialdistricting.dissolve(
            by="temp_district",
            aggfunc = {
                "COUNTYFP10": "sum",    #sum = concatenation here b/c string
                "NAME10": "count",      #kind of dummy: will be 2 only for most-recently-merged district
                "population": "sum",
                "county_id": "sum",     #should be actual sum here, kind of dummy
                "xcentr_lon": "first",  #dummy, since we'll recalculate
                "ycentr_lat": "first",
                "county_id_string": "sum", #sum = concatenation here b/c string
                "temp_district": "first"
            }
        )

        # update centroid lat/longs!
        shapef_ia_initialdistricting['xcentr_lon'] = shapef_ia_initialdistricting.centroid.x
        shapef_ia_initialdistricting['ycentr_lat'] = shapef_ia_initialdistricting.centroid.y

        #the dissolve process makes the temp_district column into the index of the dataframe
        #which then has issues when we iterate, so dump the index for a dummy one now
        shapef_ia_initialdistricting = shapef_ia_initialdistricting.reset_index(drop=True)
    
    return(shapef_ia_initialdistricting)


## Step 3: Realloacting counties until population is within bounds

#### Helper Functions for reallocation

In [5]:
# helper function: input is a shapefile with 'population' 
# column. output is a full matrix with SIGNED pop differences
def pop_diff_matrix(temp_shapefile):
    curr_n_districts = len(temp_shapefile)
    pop_diff_mat = np.zeros((curr_n_districts,curr_n_districts))
    for i in range(curr_n_districts):
        for j in range(curr_n_districts):   
            pop_diff_mat[i,j] = temp_shapefile['population'].iloc[i] - temp_shapefile['population'].iloc[j]
    
    return pop_diff_mat


## helper function: given dataframe of movable counties & dataframe 
# of the district to move to, make a list of relative distances
def calculate_rel_dist(border_counties,new_district):
    n_border_counties = len(border_counties)
    distance_list = np.zeros((n_border_counties,3))

    for i in range(n_border_counties):
        # old district distance
        oldx_dist = border_counties['xcentr_lon_2'].iloc[i] - border_counties['xcentr_lon_1'].iloc[i]
        oldy_dist = border_counties['ycentr_lat_2'].iloc[i] - border_counties['ycentr_lat_1'].iloc[i]
        distance_list[i,0] = math.sqrt(oldx_dist**2 + oldy_dist**2)

        # (potential) new district distance
        newx_dist = border_counties['xcentr_lon_2'].iloc[i] - new_district['xcentr_lon'].iloc[0]
        newy_dist = border_counties['ycentr_lat_2'].iloc[i] - new_district['ycentr_lat'].iloc[0]
        distance_list[i,1] = math.sqrt(newx_dist**2 + newy_dist**2)

        ## relative_distance. we'll move the county with highest relative distance
        distance_list[i,2] = distance_list[i,0] - distance_list[i,1]

    rel_dist = distance_list[:,2]
    return rel_dist

#helper function to find neighboring districts with the greatest population difference
def neighbor_popdiff_fun(df):
    #adjacencey matrix for 4 districts
    adj_mat_array = adj_mat_calc(df).to_numpy()   

    #SIGNED population difference array
    pop_diff_array = pop_diff_matrix(df)
    # zero out any pairs that aren't adjacent. for numpy, "*" is piecewise mult.
    neighbor_popdiff = adj_mat_array * pop_diff_array

    return neighbor_popdiff


#Identify the border counties between the two districts with biggest population difference
#return a dataframe with the border counties with a column of relative distances between 
#the big and small districts
def border_counties_df_func(shapef_ia_redist, list_of_districts, list_of_counties,scale_pop_limit_by):
    neighbor_popdiff=neighbor_popdiff_fun(shapef_ia_redist)
    #popdiff_locs is a list of ordered pairs giving the location of the
    #positive values within neighbor_popdiff corresponding to adjacent districts. Note!! Indexed on (0,n-1)!
    popdiff_locs=np.argwhere(neighbor_popdiff>0)

    big_dist_list=[]
    small_dist_list=[]
    pop_limit_switch_list=[]
    
    for i in range(popdiff_locs.shape[0]):
        big_dist_index=popdiff_locs[i][0]
        big_dist_list.append(big_dist_index)
        small_dist_index=popdiff_locs[i][1]
        small_dist_list.append(small_dist_index)
        pop_limit_switch=scale_pop_limit_by*neighbor_popdiff[big_dist_index][small_dist_index]
        pop_limit_switch_list.append(pop_limit_switch)

    df=pd.DataFrame()
    df['big_dist_index']=big_dist_list #goes from 0 to 3
    df['small_dist_index']=small_dist_list #goes from 0 to 3
    df['small_dist']=[x+1 for x in small_dist_list] #goes from 1 to 4
    df['big_dist']=[x+1 for x in big_dist_list] #goes from 1 to 4
    df['pop_limit_switch']=pop_limit_switch_list

    df=df.sort_values('pop_limit_switch', ascending=False).reset_index(drop=True)
    #Note: pop_limit_switch is a proxy for actual pop limits and we can sort off of
    #  this list. But we need to make sure other lists are sorted accordingly

    small_dist_list_for_border_counties=[]
    big_dist_counties_list=[]
    bigdist_movable_list=[]
    for j in range(popdiff_locs.shape[0]):
        # small_dist_list_for_border_counties=[x-1 for x in small_dist_list]
        small_dist_list_for_border_counties.append(list_of_districts[df.loc[j,'small_dist_index']])
        big_dist_counties_list.append(list_of_counties[df.loc[j, 'big_dist_index']])
        
        #update Nov 1
        #intersection  and .length finds the length of intersection
        #   of each county from the big district with the entirety of the small district
        #np.sign turns nonzero entries to 1s
        temp_movable_ones = np.sign(big_dist_counties_list[j].intersection(small_dist_list_for_border_counties[j].unary_union).length)
        temp_movable_boolean = temp_movable_ones.astype(bool)
        bigdist_movable_list.append(big_dist_counties_list[j].loc[temp_movable_boolean].reset_index(drop=True).copy())
        # end Nov 1 update

        #update the .intersects so that it avoids kitty corners
        # bigdist_movable_list.append(big_dist_counties_list[j].loc[big_dist_counties_list[j].intersection(small_dist_list_for_border_counties[j].unary_union)].reset_index(drop=True).copy())
        bigdist_movable_list[j]['relative_distance']=calculate_rel_dist(bigdist_movable_list[j], small_dist_list_for_border_counties[j])

    return bigdist_movable_list, df['pop_limit_switch'].tolist(), df['small_dist'].tolist(), df['big_dist_index'].tolist()

# helper function
# input is a shapefile with attributes of single counties (and columns as labeled below)
#       usually:  shapef_ia_step3
# output is a shapefile with attributes of districts
#       usually: shapef_step3_dissolved

def dissolve_by_district(county_shapefile):
    dissolved_shapefile = county_shapefile.dissolve(
        by="DISTRICT",
        aggfunc = {
            "population": "sum",
            "xcentr_lon": "first",  #dummy, since we'll recalculate
            "ycentr_lat": "first",
            #skip the county_id_string now since we aren't slicing it
            # "county_id_string": "sum", #sum = concatenation here b/c string
            "DISTRICT": "first"
        }
    )
    dissolved_shapefile['xcentr_lon'] = dissolved_shapefile.centroid.x
    dissolved_shapefile['ycentr_lat'] = dissolved_shapefile.centroid.y

    #the dissolve process has issues with index, so dump for a dummy
    dissolved_shapefile = dissolved_shapefile.reset_index(drop=True)

    return dissolved_shapefile

#gives a range for district size based off of ideal district size
def ideal_district_size_func(state_population, n_districts, tolerance):
    ideal_district_size=state_population/n_districts
    district_maximum=int(ideal_district_size*(1+tolerance))
    district_minimum=int(ideal_district_size*(1-tolerance))
    return district_minimum, district_maximum


#helper function to check for contiguity of big district as switch_func identifies a county
#to move from the big district to the small district

def big_dist_contiguity_check_func(moving_index, big_dist_df):
    df=big_dist_df.loc[big_dist_df['county_id']!=moving_index]
    big_dist_laplacian=csgraph.laplacian(adj_mat_calc(df).to_numpy())
    return  null_space(big_dist_laplacian).shape[1]


#modifying switch_threshold_func to take a list of dataframes as an argument
def switch_threshold_func(dataframe_movable_list,max_switches_threshold,switches_range_proportion):
    switches_threshold=[]
    for j in range(len(dataframe_movable_list)):
        # array with all switch counts from the movable county list
        num_switches_array = np.array(dataframe_movable_list[j]['num_switches'])

        # max and min values from the array
        max_switches = max(num_switches_array)
        min_switches = min(num_switches_array)
        # our chosen threshold for switches: halfway between max and min (floor)
        if max_switches<max_switches_threshold:
            switches_threshold.append(max_switches)
        else:
            switches_threshold.append(min_switches + np.ceil((max_switches-min_switches)*switches_range_proportion))
    return switches_threshold

# switch_func helper function
# input is a dataframe with potentially movable counties (and a count of switches)
# output is a county to switch: first priority: below threshold of switches (and population)
# Then, max relative distance of what's left
def switch_func(big_dist_list, list_of_counties, dataframe_movable_list,switches_threshold_list,pop_limit_switch_list,small_dist_list):
    for j in range(len(dataframe_movable_list)):
        dataframe_sorted = dataframe_movable_list[j].sort_values('relative_distance',ascending=False).copy()
        switches_threshold = switches_threshold_list[j]
        pop_limit_switch = pop_limit_switch_list[j]
        for i in range(dataframe_sorted.shape[0]):
            big_dist_index=big_dist_list[j]
            if (dataframe_sorted.iloc[i]['num_switches'] <= switches_threshold) & (dataframe_sorted.iloc[i]['population_2']<pop_limit_switch) &(big_dist_contiguity_check_func(dataframe_sorted.iloc[i]['county_id'],list_of_counties[big_dist_index])==1):
                return dataframe_sorted.iloc[i]['county_id'], small_dist_list[j]
                break

### Main reallocation function

In [6]:
def reallocation_func(shapef_ia_fordistricting,shapef_ia_initialdistricting
                      ,max_reallocation_limit,n_districts,n_counties,state_population, tolerance):
    # adding switch_count to county shapefile which we will be using
    shapef_counties_for_realloc = shapef_ia_fordistricting.copy()

    # make a column with num_switches
    shapef_counties_for_realloc['num_switches'] = np.zeros(shapef_counties_for_realloc.shape[0],dtype=int)

    shapef_ia_redist = shapef_ia_initialdistricting.drop(columns=['NAME10', 'COUNTYFP10','county_id','temp_district']).copy()

    # make a column with a district #, 1-4
    shapef_ia_redist['district_label'] = shapef_ia_redist.index + 1

    for k in range(max_reallocation_limit):
        # Making separate geodataframes for each district (one district/attribute in each).
        list_of_districts=[]
        for i in range(n_districts):
            list_of_districts.append(shapef_ia_redist.iloc[[i]].reset_index(drop=True))

        #the identity overlay takes district n and splits it up by county
        list_of_counties=[]
        for i in range(n_districts):
            list_of_counties.append(list_of_districts[i].overlay(shapef_counties_for_realloc,how='identity',keep_geom_type=True))

        #this records the district number each county is assigned to in the first pass
        shapef_counties_for_realloc['DISTRICT']=''
        for i in range(n_counties):
            for j in range(n_districts):
                # if i<j:
                if shapef_counties_for_realloc.iloc[i]['COUNTYFP10'] in list_of_counties[j]['COUNTYFP10'].tolist():
                    shapef_counties_for_realloc.loc[i,'DISTRICT']=j+1

        border_counties_df_func_outputs=border_counties_df_func(shapef_ia_redist, list_of_districts, list_of_counties,scale_pop_limit_by)
        bigdist_movable_list=border_counties_df_func_outputs[0]
        pop_limit_switch_list=border_counties_df_func_outputs[1]
        small_district_list = border_counties_df_func_outputs[2]
        big_district_list = border_counties_df_func_outputs[3]
        
        # bigdist_movable
        switches_threshold_list=switch_threshold_func(bigdist_movable_list, max_switches_threshold,switches_range_proportion)
        switch_func_output=switch_func(big_district_list,list_of_counties, bigdist_movable_list, switches_threshold_list,pop_limit_switch_list,small_district_list)
        movingcounty_index=switch_func_output[0]

        # Update 99 row dataframe:
        shapef_counties_for_realloc.loc[movingcounty_index,'DISTRICT'] = switch_func_output[1]
        shapef_counties_for_realloc.loc[movingcounty_index,'num_switches'] = shapef_counties_for_realloc.loc[movingcounty_index,'num_switches']+1

        # overwriting old 4 row geodataframe with the new version (dissolved based on updated district number)
        shapef_ia_redist = dissolve_by_district(shapef_counties_for_realloc)
        
        district_min=ideal_district_size_func(state_population, n_districts, tolerance)[0]
        district_max=ideal_district_size_func(state_population, n_districts, tolerance)[1]

        if shapef_ia_redist['population'].max() in range(district_min, district_max+1) and shapef_ia_redist['population'].min() in range(district_min, district_max+1):
            break

    return shapef_counties_for_realloc, shapef_ia_redist

## Compactness measures and drawing district maps

Princeton Gerrymandering Project uses Reock (minimum bounding circle) and Polsby-Popper (perimeter) in their report cards
https://gerrymander.princeton.edu/redistricting-report-card-methodology

Polsby-Popper: $PP = \frac{\text{Area of district}}{\text{Area of circle with same perimeter as district}} = 4\pi \left(\frac{\text{Area of district}}{\text{(Perimeter of district)}^2}\right) $

Reock: $R = \frac{\text{Area of district}}{\text{Area of MBC}}$

(other metrics described here: https://fisherzachary.github.io/public/r-output.html)


(Skipping convex hull stuff (for now), but examples are at end of Linear_Programming/Iowa_redistricting_miniset_for_perim.ipynb)

In [7]:
#this is computes compactness scores and appends them as columns to the shapef_ia_redist df
def compactness_func(shapef_ia_redist):
    shapef_ia_compactness = shapef_ia_redist.copy()
    shapef_ia_compactness['area'] = shapef_ia_compactness['geometry'].area
    shapef_ia_compactness['perimeter'] = shapef_ia_compactness['geometry'].length
    #Polsby-Popper Score
    shapef_ia_compactness['PolsbyPopper']=4*math.pi*shapef_ia_compactness['area']/(shapef_ia_compactness['perimeter']**2)
    #radius of minimum bounding circle
    shapef_ia_compactness['min_bounding_radius']=shapef_ia_compactness['geometry'].minimum_bounding_radius()
    #Reock Score
    shapef_ia_compactness['Reock']=shapef_ia_compactness['area']/(math.pi*(shapef_ia_compactness['min_bounding_radius']**2))
    
    return shapef_ia_compactness

color_dict = { 1 : '#3995ff',
               2 : '#ff8539',
               3 : '#ffe839',
               4 : '#d139ff',
               }

def distmap_by_county(map_data,data_label):
    plot_distmap = (
        ggplot(map_data)
    + geom_map(aes(fill='DISTRICT')
        ,show_legend=True
        )
    + geom_label(aes(x='xcentr_lon', y='ycentr_lat', label=data_label,size=2)
        , show_legend=False)
    + theme_minimal()
    + theme(axis_text_x=element_blank(),
            axis_text_y=element_blank(),
            axis_title_x=element_blank(),
            axis_title_y=element_blank(),
            axis_ticks=element_blank(),
            panel_grid_major=element_blank(),
            panel_grid_minor=element_blank(),
            plot_background = element_rect(fill = 'white')       #whole png area
            )
    + scale_fill_manual(values=color_dict)  #require district i to always be color i
    )
    return plot_distmap

# Generate district maps based on random seed

In [8]:
# setting thresholds for helper functions
scale_pop_limit_by=6
max_switches_threshold=5
switches_range_proportion=.75
n_counties = 99
n_districts = 4
tolerance = .015
max_reallocation_limit = 120  #for loop max on step 3

In [9]:
for seed_number in range(100):
    random.seed(seed_number)
    shapef_ia_initialdistricting = initial_alloc_func(shapef_ia_fordistricting)
    reallocation_output = reallocation_func(shapef_ia_fordistricting,shapef_ia_initialdistricting
                                        ,max_reallocation_limit,n_districts,n_counties,state_population, tolerance)
    shapef_counties_for_realloc = reallocation_output[0]
    shapef_ia_redist = reallocation_output[1]
    shapef_ia_compactness=compactness_func(shapef_ia_redist)  

    district_map=distmap_by_county(shapef_counties_for_realloc,'county_id')

    #all the exports
   
    #kth iteration of generating maps. Change 0 to k for future use
    iter_number=str(seed_number).rjust(3,'0')

     #export image of district as png
    district_map.save("district_maps/seed_{}_district_map.png".format(iter_number))

    #exporting dataframe with 99 county gpd with district allocation for winner analysis
    #Calculate winners with winner_tabulation() in a separate notebook for each file in ./allocation_by_county
    shapef_counties_for_realloc.to_csv('allocation_by_county/{}.csv'.format('seed_'+str(iter_number)+'_by_county'), index=False, header=True)

    #exporting dataframe with district gpd for compactness analysis. Pull Polsby-Popper and Reock scores for analysis in a separate notebook
    shapef_ia_compactness.to_csv('allocation_by_district/{}.csv'.format('seed_'+str(iter_number)+'_by_district'), index=False, header=True)

/opt/homebrew/Caskroom/miniforge/base/envs/myenv/lib/python3.10/site-packages/plotnine/ggplot.py:587: PlotnineWarning: Saving 6.4 x 4.8 in image.
/opt/homebrew/Caskroom/miniforge/base/envs/myenv/lib/python3.10/site-packages/plotnine/ggplot.py:588: PlotnineWarning: Filename: district_maps/seed_000_district_map.png
/opt/homebrew/Caskroom/miniforge/base/envs/myenv/lib/python3.10/site-packages/plotnine/ggplot.py:587: PlotnineWarning: Saving 6.4 x 4.8 in image.
/opt/homebrew/Caskroom/miniforge/base/envs/myenv/lib/python3.10/site-packages/plotnine/ggplot.py:588: PlotnineWarning: Filename: district_maps/seed_001_district_map.png
/opt/homebrew/Caskroom/miniforge/base/envs/myenv/lib/python3.10/site-packages/plotnine/ggplot.py:587: PlotnineWarning: Saving 6.4 x 4.8 in image.
/opt/homebrew/Caskroom/miniforge/base/envs/myenv/lib/python3.10/site-packages/plotnine/ggplot.py:588: PlotnineWarning: Filename: district_maps/seed_002_district_map.png
/opt/homebrew/Caskroom/miniforge/base/envs/myenv/lib/py

Ran on 12/6 for seed_number in range(5) and it finished in 10m 38.0s. Ran up to seed 64 in 507 minutes

Ran on 12/7 for seeds in range(16,100) for switch_threshold=5